In [2]:
!pip install transformers
!pip install sentencepiece
!pip install nltk
!pip install ipywidgets
!pip install tensorboard

In [4]:
import pandas as pd
import numpy as np
import random
import joblib
import torch
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import model_selection
# from DataAugmentation 
from data_utils import loadDatafromFile,createTokenizedDf,CompDataset,createkfoldData,createDataloaders
from data_utils_unlabelled import loadUnlabelledDatafromFile,createTokenizedDfUnlabelled,CompDatasetUnlabelled
from transformers import AdamW
from importlib import reload
import config
from transformers import XLMRobertaForTokenClassification, XLMRobertaConfig ,BertModel, XLMRobertaTokenizer, XLMRobertaModel, BertForTokenClassification
from seqeval.metrics import accuracy_score, classification_report
from sklearn.metrics import matthews_corrcoef,classification_report,f1_score
import engine
# from model_new import EntityModel
import torch.nn as nn
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from  loss_functions import ClassificationCost, ConsistencyCost, update_teacher_params
from utils_functions import set_seed, set_seed_random, sigmoid_rampup, get_consistency_weight, EarlyStopping
from model_xlmroberta import EntityModel , GaussianNoise

In [5]:
# import data_utils
# data_utils = reload(data_utils)
# from data_utils import loadDatafromFile,createTokenizedDf,CompDataset,createkfoldData,createDataloaders
# import data_utils_unlabelled
# data_utils_unlabelled = reload(data_utils_unlabelled)
# from data_utils_unlabelled import loadUnlabelledDatafromFile,createTokenizedDfUnlabelled,CompDatasetUnlabelled, createkfoldData,createDataloaders

In [6]:
model = EntityModel(with_noise_layer = False, dropout_layer=True) # init model
model.cuda()
model.eval()

In [7]:
def process_data_labelled(filePath_src_eval,filePath_tar_eval, filePath_srcTags_eval,filePath_tarTags_eval,model_type='xlm',seed=123):
    
    set_seed_random(seed)
#     print(seed)
    dataObjEval = loadDatafromFile(filePath_src_eval,filePath_tar_eval, filePath_srcTags_eval,filePath_tarTags_eval)
    df_test = dataObjEval.createDf()
    df_test= df_test.sample(n = 250) # sample randomly 250 data
    df_test = df_test.reset_index(drop=True)
    
    obj_tokenized_test = createTokenizedDf(df_test,model_type)

    df_new_test = obj_tokenized_test.convertDf()

    test_data = CompDataset(df_new_test,model_type)
    
    return test_data,df_test

In [8]:
next(iter(test_dataloader))

NameError: name 'test_dataloader' is not defined

In [34]:
# # Read the labelled data
# def process_data_labelled(filePath_src,filePath_tar, filePath_srcTags,filePath_tarTags,
#                  filePath_src_eval,filePath_tar_eval, filePath_srcTags_eval,filePath_tarTags_eval,
#                  model_type='xlm'):
    
    
#     dataObjTrain = loadDatafromFile(filePath_src,filePath_tar, filePath_srcTags,filePath_tarTags)
#     df_train= dataObjTrain.createDf() # get dataframe from files    
#     df_train = df_train.sample(frac=1).reset_index(drop=True)
#     df_train=df_train.iloc[:500,:]
#     dataObjEval = loadDatafromFile(filePath_src_eval,filePath_tar_eval, filePath_srcTags_eval,filePath_tarTags_eval)
#     df_eval = dataObjEval.createDf()
#     obj_tokenized_train = createTokenizedDf(df_train,model_type)
#     obj_tokenized_test = createTokenizedDf(df_eval,model_type)
#     df_new_train = obj_tokenized_train.convertDf()
#     df_new_eval = obj_tokenized_test.convertDf()
#     train_data = CompDataset(df_new_train,model_type)
#     test_data = CompDataset(df_new_eval,model_type)
#     return train_data,test_data


In [9]:
engine=reload(engine)

In [10]:
from random import randint
test_sets = 100
lst = []
while True:
    
    if len(lst) == test_sets:
        break
    ran_nr = randint(1,1000)
    if ran_nr not in lst:
        lst.append(ran_nr)

print(len(set(lst)))
print(lst[:10])

100
[985, 129, 735, 85, 919, 38, 229, 944, 100, 180]


In [11]:
## set_seed()
# generate some random numbers for setting seed
lab_data = input('labeled data :')
unlabeld_data = input('unlabel data :')
sup_model_name = f'../models/training_data/supervised_with_{lab_data}_labeled_data.bin'
teacher_model_name = f'../models/pseudolabeling/TeacherModels/teacherModel_v1_{lab_data}_label_{unlabeld_data}_unlabel_pseudolabeling_confidence_0.6.bin'
student_model_name = f'../models/pseudolabeling/StudentModels/studentModel_v1_{lab_data}_label_{unlabeld_data}_unlabel_pseudolabeling_confidence_0.6.bin'

lst_sup_mcc_score = []
lst_st_mcc_score = []
lst_te_mcc_score = []
tot_mcc_score_sup = 0 
tot_mcc_score_st  = 0 
tot_mcc_score_te  = 0 

for item in tqdm(lst) :
        
        test_data, df_test= process_data_labelled(config.filePath_src_eval,config.filePath_tar_eval, config.filePath_srcTags_eval,config.filePath_tarTags_eval,seed=item)
        loader_obj = createDataloaders(test_data,config.TRAIN_BATCH_SIZE)
        test_dataloader = loader_obj.createDataloaders()
        
        model.load_state_dict(torch.load(sup_model_name))
        test_metrics_sup = engine.eval_fn(test_dataloader, model)
        tot_mcc_score_sup+=test_metrics_sup[4]
        lst_sup_mcc_score.append(test_metrics_sup[4])
        
        model.load_state_dict(torch.load(student_model_name))
        test_metrics_st = engine.eval_fn(test_dataloader, model)
        tot_mcc_score_st+=test_metrics_st[4]
        lst_st_mcc_score.append(test_metrics_st[4])
        
        model.load_state_dict(torch.load(teacher_model_name))
        test_metrics_te = engine.eval_fn(test_dataloader, model)
        tot_mcc_score_te+=test_metrics_te[4]
        lst_te_mcc_score.append(test_metrics_te[4])

avg_mcc_score_sup = tot_mcc_score_sup/test_sets
avg_mcc_score_st = tot_mcc_score_st/test_sets
avg_mcc_score_te = tot_mcc_score_te/test_sets

print(avg_mcc_score_sup, avg_mcc_score_st, avg_mcc_score_te)
        
        
# # print(next(iter(test_dataloader)))
        



labeled data : 250
unlabel data : 1750


  0%|          | 0/100 [00:00<?, ?it/s]

[2022-04-28 13:19:00.148 pytorch-1-6-gpu-py3-ml-g4dn-xlarge-c630bdb4e3ad8d68ab6e5727a214:6173 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-04-28 13:19:00.318 pytorch-1-6-gpu-py3-ml-g4dn-xlarge-c630bdb4e3ad8d68ab6e5727a214:6173 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


INFO:numexpr.utils:NumExpr defaulting to 4 threads.
/root/Thesis/src/code/code_files/engine.py:411: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np_arr_labels = np.array(tar_labels)
INFO:numexpr.utils:NumExpr defaulting to 4 threads.
/root/Thesis/src/code/code_files/engine.py:411: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np_arr_labels = np.array(tar_labels)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
INFO:

0.2544090890271343 0.2842176588623416 0.2868080336804173


In [121]:
print(avg_mcc_score_sup, avg_mcc_score_st, avg_mcc_score_te)
print(tot_mcc_score_sup,tot_mcc_score_st,tot_mcc_score_te)

0.26717766131965315 0.27675224050133407 0.27212297523513823
26.717766131965313 27.675224050133405 27.212297523513826


In [124]:
flag=0
for i in range(len(lst_sup_mcc_score)):
    if lst_st_mcc_score[i] > lst_sup_mcc_score[i]:
        flag+=1
stat_impr_st = flag
flag=0
for i in range(len(lst_sup_mcc_score)):
    if lst_te_mcc_score[i] > lst_sup_mcc_score[i]:
        flag+=1
stat_impr_te = flag
print(stat_impr_st,stat_impr_te)

76 68


In [110]:
# print(lst_sup_mcc_score)
# print(lst_st_mcc_score)
# print(lst_te_mcc_score)
# dict_scores = dict()
dict_scores['1750'] = {'supervised': avg_mcc_score_sup,'student': avg_mcc_score_st,'teacher': avg_mcc_score_te, 'stat_impr_st': stat_impr_st, 'stat_impr_te':stat_impr_te}
dict_scores

{'250': {'supervised': 0.25374236207164924,
  'student': 0.28054619538886877,
  'teacher': 0.28345596855153105,
  'stat_impr_st': 98,
  'stat_impr_te': 99},
 '500': {'supervised': 0.2883729841953051,
  'student': 0.29909402295207177,
  'teacher': 0.3011482964085686,
  'stat_impr_st': 79,
  'stat_impr_te': 92},
 '750': {'supervised': 0.3136512725272238,
  'student': 0.31960919324374326,
  'teacher': 0.31265870227711556,
  'stat_impr_st': 75,
  'stat_impr_te': 46},
 '1000': {'supervised': 0.32204379465700816,
  'student': 0.34294089336620415,
  'teacher': 0.3456469100069271,
  'stat_impr_st': 92,
  'stat_impr_te': 96},
 '1250': {'supervised': 0.33535159488242405,
  'student': 0.33903866673395294,
  'teacher': 0.3435257877380865,
  'stat_impr_st': 61,
  'stat_impr_te': 78},
 '1500': {'supervised': 0.3343286622625563,
  'student': 0.35057468379453505,
  'teacher': 0.35143383716584514,
  'stat_impr_st': 94,
  'stat_impr_te': 94},
 '1750': {'supervised': 0.328103515750323,
  'student': 0.354

In [117]:
import json
jsonString = json.dumps(dict_scores,indent=4) # dump open pr data to json file
jsonFileobj = open("scores_test_mt.json", "w")
jsonFileobj.write(jsonString)
jsonFileobj.close()

In [112]:
print(sorted(lst_sup_mcc_score))
print(sorted(lst_st_mcc_score))
print(sorted(lst_te_mcc_score))


[0.2704628842143567, 0.28256790199227366, 0.28991584834955486, 0.29043290686164935, 0.2944057598516638, 0.2980118258895146, 0.29935202537119143, 0.30068403542015304, 0.30480684617412945, 0.30521315118954045, 0.3062601233482304, 0.3067115053644888, 0.30708265447983557, 0.30773636332067195, 0.3093157533529955, 0.30986958506437984, 0.3121733158437746, 0.31291671867698867, 0.31445568451918143, 0.3146991823677846, 0.3149679995822772, 0.31536886068028885, 0.31619733305139675, 0.31663682984334224, 0.3184424779547067, 0.3185876474918324, 0.3188093628468542, 0.3188498017075039, 0.32077366559512505, 0.320858855932626, 0.32222925860118884, 0.32224992212038955, 0.322585644057695, 0.3229711864344981, 0.32341771775103073, 0.32343224306686535, 0.32365398340338697, 0.32390582825898306, 0.3244242072763923, 0.32455420559212483, 0.324684093061665, 0.32513104643238727, 0.3255800237813895, 0.32606849038036634, 0.32636035607339414, 0.3264822803899133, 0.3269900780694534, 0.3270309086510518, 0.32703663210344